In [223]:
import torch
from transformers import GPT2LMHeadModel

In [224]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [225]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>') 
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [226]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [227]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


In [228]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [229]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [230]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [231]:
import urllib.request

# urllib.request.urlretrieve(
#     "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
#     filename="ChatBotData.csv",
# )
Chatbot_Data = pd.read_csv("ChatBotData.csv")
# Test 용으로 300개 데이터만 처리한다.
Chatbot_Data = Chatbot_Data[:300]
Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [232]:
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len = 40):
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER
        
    def __len__(self):
        return len(self._data)
    
    def __getitem__(self, idx):
        turn = self._data.iloc[idx]  #dataframe에서 idx행에 있는 데이터
        q = turn["Q"] # 질문을 가져온다
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다
        
        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.
        
        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        
        #print("q_toked : ",q_toked)
        # q_toked :  ['<usr>', '▁군대', '▁언제', '▁끝나', '나', '<unused1>']
        
        q_len = len(q_toked)
        #print(q_len)
        
        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        
        #print("a_toked : ", a_toked)
        # a_toked : ['<sys>', '▁다음', '▁달', '에는', '▁더', '▁절약', '해', '봐', '요', '▁', '</s>']
        
        a_len = len(a_toked)
        
        #질문의 길이가 최대 길이(gpt seq-length)보다 크면
        if q_len > self.max_len:
            # a_len = self.max_len - q_len   #이거 두개는 왜 넣은지 모르겠음
            # if a_len <= 0:
            q_toked = q_toked[-(int(self.max_len/2)):] #이거한번 출력해보기 이러면 처음 self.q_token 붙여둔건 날아가지않을까 
            
            #print("q_toked half : ", q_toked)
            #q_toked half :  ['야', '▁마', '바', '사', '▁부를', '거', '야', '▁아', '자', '차', '카', '▁부를', '거', '야', '▁타파', '하', '▁부를', '거', '야', '<unused1>']
            
            q_len = len(q_toked)
            #print("q_len", q_len)
            #print("a_len", a_len)
            
            a_len = self.max_len - q_len
            a_token = a_toked[:a_len]    # 이러면 self.eos는 날아가지 않을까?     #이거 len도 잘 고쳐야할 듯
            #print("a_len", a_len)
            #print("a_token : ", a_toked)
            a_len = len(a_token)
                
        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len
            if a_len <=0:
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                #print("q_toked half : ", q_toked)
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            
            #print("a_token : ", a_toked)
            a_len = len(a_toked)
            
            
        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,]*q_len + a_toked[1:] # a_toked와 q_toked 한번 출력해봐야 겠다.
        # print(labels)
        # ['<unused0>', '<unused0>', '<unused0>', '<unused0>', '<unused0>', '<unused0>', '<unused0>', '▁하루', '가', '▁또', '▁가', '네', '요', '▁', '</s>']
        
        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        #print("q_len", q_len)
        #print("a_len", a_len)
        mask = [0]*q_len + [1]*a_len + [0]*(self.max_len - q_len - a_len)
        #print(mask)
        # [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0]  <- max_len(40)개
        
        # 답변 labels를 index로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        #print(labels_ids)
        # [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 29080, 14553, 10341, 9122, 27511, 8084, 1]
        
        #최대길이만큼 패딩
        while len(labels_ids) < self.max_len:  #이거 그냥 빼기로 수정하는게 좋을거 같음
            labels_ids += [self.tokenizer.pad_token_id]
            
        #print(labels_ids)
        # [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 29080, 14553, 10341, 9122, 27511, 8084, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
            
        # 질문 + 답변을 index로 만든다
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        
        #print(token_ids)
        # 이거 앞에 2가 있어야 되나 싶기도 하고
        #[6853, 7991, 9050, 8159, 8335, 8429, 18705, 6853, 7991, 39153, 8702, 18705, 6853, 7991, 29918, 12557, 18705, 6853, 7991, 10, 4, 29080, 14553, 10341, 9122, 27511, 8084, 1]
        #[2, 12396, 9136, 7380, 11355, 7058, 6853, 15495, 13358, 7182, 10, 4, 18372, 9267, 15730, 8711, 9633, 6853, 27511, 8084, 1]  
        
        # 최대길이만큼 패딩
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]
            
        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)
    
    # token_ids는 질문+감정+답변+pad_token_id 순이라는데 출력해보자
    # mask는 질문 q가 들어 가는 곳에는 0, 답변 a가 위치한 곳에는 1, 빈공간은 0으로 채워진다
    # labels는 질문의 길이만큼 mask 문자 그리고 답변 a의 id


In [233]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [234]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_set = ChatbotDataset(Chatbot_Data, max_len=40)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2

generator = torch.Generator()
generator.manual_seed(2009)
train_sampler = torch.utils.data.RandomSampler(train_set, generator=generator)

train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,pin_memory=True)

In [235]:
# print("start")
# for batch_idx, samples in enumerate(train_dataloader):
#     token_ids, mask, label = samples
#     # print("token_ids ====> ", token_ids)
#     # print("mask =====> ", mask)
#     # print("label =====> ", label)
# print("end")

# ---------- 학습부분 -----------

In [236]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
model.train()

learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18

In [237]:
print(torch.cuda.is_available())

True


In [238]:
print ("start")
for epoch in range(epoch):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

start


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

# ------- 테스트 -----------

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + sent + A_TKN + a)).unsqueeze(dim=0)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))